## scoring
### for scoring districting plans

In [1]:
from gerrychain import Graph, Partition, Election
from gerrytools.scoring import *
import pandas as pd
import geopandas as gpd

In [ ]:
%load_ext autoreload
%autoreload 2

All of our scores are functions that take a GerryChain `Partition` and produce either a numerical (plan-wide) score or a mapping from district or election IDs to numeric scores. For our examples, we will use a 2020 Maryland VTD shapefile to build our underlying dual graph, since the shapefile has demographic and electoral information that our scores will rely on.

In [ ]:
%%time
graph = Graph.from_file("data/MD_vtd20/")

In [ ]:
elections = ["PRES12", "SEN12", "GOV14", "AG14", "COMP14", 
             "PRES16", "SEN16", "GOV18", "SEN18", "AG18", "COMP18"]

# use our list of elections ablve to create `Election` updaters for each contest
# Ex: in our shapefile, the column `PRES12R` refers to the votes Mitt 
# Romney (R) received in the 2012 Presidential general election
updaters = {}
for e in elections:
    updaters[e] = Election(e, {"Dem": e+"D", "Rep": e+"R"})

The `demographic_updaters()` function returns a dictionary of `Tally` updaters that track the number of people of a given demographic group. You can pass as a list with as many demographic groups as you wish (example below):

In [ ]:
demographic_updaters(["TOTPOP20", "VAP20"])

In [ ]:
# add updaters that track total population, total voting age population, 
# and Black and Hispanic voting age population
updaters.update(demographic_updaters(["TOTPOP20", "VAP20", "BVAP20", "HVAP20"]))

# create the partition on which we'll generate scores
# since `MD_CD_example.csv` is a CSV with `GEOID20` -> district assignment,
# we need to replace the `GEOID20`s with integer node labels to match the graph's nodes.
geoid_to_assignment = pd.read_csv("data/MD_CD_example.csv", header=None).set_index(0).to_dict()[1]
assignment = {n: geoid_to_assignment[graph.nodes[n]["GEOID20"]] for n in graph.nodes}
partition = Partition(graph, assignment, updaters)

### partisan scores
All our partisan scores require at least a list of elections (we'll use our `elections` list defined above). Some of them additionally require the user to specify a POV party (in our case, either `Dem` or `Rep`). All of these partisan scores return a dictionary that maps election names to the score for that election; it is up to the user to aggregate (often by summing or averaging) the scores across every election. For a simple example, let's use the score function that returns the number of Democratic seats won in each election.

In [ ]:
seats(elections, "Dem")

Note that the output of `seats(elections, "Dem")` is of type `Score`, which functions like a Python `namedtuple`: for any object `x` of type `Score`, `x.name` returns the name of the score, and `x.apply` returns a function that takes a `Partition` as input and returns the score. See below:

In [ ]:
seats(elections, "Dem").name

In [ ]:
seats(elections, "Dem").apply(partition)

Note that we can easily find the number of Republican seats like so:

In [ ]:
seats(elections, "Rep").apply(partition)

Moreover, we can pass `mean=True` to return the average of the score over all elections, rather than a dictionary:

In [ ]:
seats(elections, "Rep", mean=True).apply(partition)

Some partisan scores (`mean_median`, `efficiency_gap`, `partisan_bias`, `partisan_gini`) do not require the user to specify the POV party in the call. This is not because there isn't a POV party, but because these functions call GerryChain functions that automatically set the POV party to be the **first** party listed in the updater for that election. Since we always list `Dem` first in this notebook, this means `Dem` will be the POV party for these scores— but this is something you should keep in mind when setting up your updaters and your partition.

In [ ]:
# Positive values denote an advantage for the POV party
efficiency_gap(elections).apply(partition)

If you know you want to use a lot of scores, it can be helpful to make a list of the scores of interest, like so:

In [ ]:
partisan_scores = [
    seats(elections, "Dem"),
    seats(elections, "Rep"),
    signed_proportionality(elections, "Dem", mean=True),
    absolute_proportionality(elections, "Dem", mean=True),
    efficiency_gap(elections, mean=True),
    mean_median(elections),
    partisan_bias(elections),
    partisan_gini(elections),
    # Note that `eguia` takes several more arguments — see the documentation for more details
    eguia(elections, "Dem", graph, updaters, "COUNTYFP20", "TOTPOP20"),
]

Now, we can make use of the `summarize()` function to evaluate all the scores on this partition:

In [ ]:
partisan_dictionary = summarize(partition, partisan_scores)
partisan_dictionary["mean_median"]

In [ ]:
partisan_dictionary["mean_efficiency_gap"]

### demographic scores

Our demographic scores return a dictionary that maps districts to demographic information, either population counts or shares.

In [ ]:
# `demographic_tallies()` takes a list of the demographics you'd like to tally
tally_scores = demographic_tallies(["TOTPOP20", "BVAP20", "HVAP20"])
tally_dictionary = summarize(partition, tally_scores)
tally_dictionary

In [ ]:
# `demographic_shares()` takes a dictionary where each key is a total demographic column
# that will be used as the denominator in the share (usually either `TOTPOP20` or `VAP20`)
# and each value is a list of demographics on which you'd like to compute shares
share_scores = demographic_shares({"VAP20": ["BVAP20", "HVAP20"]})
share_dictionary = summarize(partition, share_scores)
share_dictionary

#### Two things to note:

Both `demographic_tallies()` and `demographic_shares()` return _lists_ of `Score`s (one for each demographic of interest), so if we want to just score one demographic, we'd have to index into the list in order to call `.function()`:

In [ ]:
demographic_tallies(["BVAP20"])[0].apply(partition)

Moreover, you can only use these scores on demographic columns that have already been tracked as `Tally` updaters when we instantiated our partition. If you try a new column (say, `WVAP20`) things won't work!

In [ ]:
demographic_tallies(["WVAP20"])[0].apply(partition)

Our last demographic updater is `gingles_districts()`, which takes in a dictionary of the same type as `demographic_tallies()` as well as a `threshold` between 0 and 1. Just like the other two demographic scores it returns a list of `Score`s, but here the `Score`s represent the number of districts where the demographic group's share is above the `threshold`. (When the threshold is 0.5 — the default — these districts are called _Gingles' Districts_.

In [ ]:
gingles_scores = gingles_districts({"VAP20": ["BVAP20", "HVAP20"]}, threshold=0.5)
gingles_dictionary = summarize(partition, gingles_scores)
gingles_dictionary

### compactness scores

We can count the number of county _splits_ (the number of counties that are assigned to more than one district) as well the number of county _pieces_ (the sum of the number of unique $($_county_, _district_$)$ pairs over every split county).

By passing a column name to the `pop_col` keyword argument, you can specify whether you just want splits and pieces that impact population. 

In [ ]:
# if we had a column in our data defining municipal boundaries, we could
# similarly county municipal splits/pieces
county_scores = [
    splits("COUNTYFP20"),
    pieces("COUNTYFP20"),
]
county_dictionary = summarize(partition, county_scores)
county_dictionary

##  compactness scores

We can also score for compactness in a variety of different ways. 

The functions `reock`, `polsby_popper`, `schwartzberg`, `convex_hull`, and `pop_poplygon`, get each of these scores for each district in a plan. Unlike the other scoring functions, each of these takes a GeoDataFrame and a crs as arguments. Most of these require the use of a pre-dissolved GeoDataFrame by plan district. This is so the geometries can be used for calculations. 

Below, we go through the call to each of these functions, which all take similar arguments.

In [9]:
md_example = pd.read_csv("data/MD_CD_example.csv", header=None).rename(columns={0:"GEOID20", 1: "assignment"})
vtd_gdf = gpd.read_file("data/MD_vtd20")
dissolved_gdf = gpd.GeoDataFrame(md_example.merge(vtd_gdf, on = "GEOID20"), geometry="geometry").dissolve(by="assignment", aggfunc="sum").reset_index()

In [ ]:
compactness_scores = [reock(dissolved_gdf, dissolved_gdf.crs),
                      polsby_popper(dissolved_gdf, dissolved_gdf.crs), 
                      schwartzberg(dissolved_gdf, dissolved_gdf.crs),
                      convex_hull(dissolved_gdf, dissolved_gdf.crs, assignment_col="assignment"), 
                      pop_polygon_dict = pop_polygon(vtd_gdf, dissolved_gdf, dissolved_gdf.crs, pop_col="TOTPOP20", assignment_col="assignment")]
compactness_dictionary = summarize(compactness_scores, partition)
compactness_dictionary

# summary

We can string together all of our score lists and use them to fully summarize our partition:

In [ ]:
all_scores = partisan_scores + tally_scores + share_scores + gingles_scores + county_scores + compactness_scores
summary_dictionary = summarize(partition, all_scores)
summary_dictionary

## misc

Other miscellaneous scores we can use:

In [ ]:
# max_deviation() gives us the maximum deviation from ideal district population, either as a count or as a percent
max_deviation("TOTPOP20").apply(partition)

In [ ]:
max_deviation("TOTPOP20", pct=True).apply(partition)